In [3]:
import duckdb
import pandas as pd
import pandas_gbq
import re
import time
import numpy as np
import os
from utils import generate_filename, update_bq_table
from dotenv import load_dotenv

load_dotenv('../myenv.env')

BUCKET_NAME = os.getenv("BUCKET_NAME")
PROJECT_ID = os.getenv("PROJECT_ID")
GCP_KEY_ID = os.getenv("GCP_KEY_ID")
GCP_ACCESS_KEY = os.getenv("GCP_ACCESS_KEY")
DATASET = "nhanes"

bucket_name = 'nhanes_clean'

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [6]:
def configure_duckdb_gcp():
    """
    Configures DuckDB for use with GCP by setting necessary S3 configurations.
    Retrieves GCP access keys from environment variables.
    Raises EnvironmentError if keys are not found.
    """
    gcp_access_key_id = GCP_KEY_ID
    gcp_secret_access_key = GCP_ACCESS_KEY

    if not gcp_access_key_id or not gcp_secret_access_key:
        logging.error("GCP access key or secret key not set in environment variables.")
        raise EnvironmentError("GCP keys not found in environment variables.")

    duckdb.sql("SET s3_endpoint='storage.googleapis.com';")
    duckdb.sql(f"SET s3_access_key_id='{gcp_access_key_id}';")
    duckdb.sql(f"SET s3_secret_access_key='{gcp_secret_access_key}';")

In [4]:
%sql duckdb:///:default:

In [22]:
def get_metadata_df():
    """
    Retrieves metadata dataframe from BigQuery.

    Returns:
        DataFrame containing metadata information.
    """
    query = f"""
    SELECT *,
        replace(gcs_data_filename,'.XPT','.parquet') as parquet_filename
    FROM `{PROJECT_ID}.nhanes.nhanes_file_metadata`
    WHERE dataset = 'all-continuous-nhanes'
    AND page_component != 'Limited Access'
    """
    return pd.read_gbq(query, 
                       project_id=PROJECT_ID,
                       dialect="standard")

In [23]:
df = get_metadata_df()

In [21]:
# df

In [3]:
### GET METADATA DATAFRAME
metadata_df = pd.read_gbq(
    """SELECT *,
    replace(gcs_data_filename,'.XPT','.parquet') as parquet_filename
    FROM nhanes.nhanes_file_metadata
    WHERE dataset = 'all-continuous-nhanes'
    AND page_component != 'Limited Access'
    """,
    project_id="nhanes-genai",
    dialect="standard",
)

In [100]:
# metadata_df

In [4]:
file_subset_df = metadata_df[["data_file_name","page_component"]].drop_duplicates()
file_subset_df["data_file_name"] = file_subset_df["data_file_name"].str.strip()
file_subset_df["page_component"] = file_subset_df["page_component"].str.strip()

file_subset_df.drop_duplicates(inplace=True)


In [103]:
# file_subset_df.sort_values(by=['data_file_name','page_component'])[file_subset_df['data_file_name'].str.contains('Acrylamide')]['data_file_name'].unique().tolist()


/var/tmp/ipykernel_3956/2600076760.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  file_subset_df.sort_values(by=['data_file_name','page_component'])[file_subset_df['data_file_name'].str.contains('Acrylamide')]['data_file_name'].unique().tolist()


['Acrylamide & Glycidamide', 'Acrylamide & Glycidamide - Special Sample']

,Success


In [25]:
# %%sql
# output_df << SELECT * FROM read_parquet('s3://nhanes_clean/all-continuous-nhanes/data/acculturation_questionnaire_*.parquet',
#                                            union_by_name=True,
#                                            filename=True);

In [34]:
# duckdb.rollback()

In [7]:
configure_duckdb_gcp()

In [15]:
# df = duckdb.read_parquet('s3://nhanes_clean/all-continuous-nhanes/data/dual_energy_x_ray_absorptiometry_whole_body*.parquet',
#                     union_by_name=True,
#                     filename=True).to_df()

In [14]:
# df['filename'].unique().tolist()

In [46]:
# df['SEQN'] = df['SEQN'].astype('int64')

In [51]:
# for column in df.columns.tolist():
#     if column != 'filename':
#         df[column] = df[column].astype('int64')

In [ ]:
# alias = generate_filename("Dietary Supplement Use 30-Day - File 1,".lower() + " " + page_component.lower(),'')
# alias

In [55]:
# metadata_df['data_file_name'].unique().tolist()

In [96]:
# metadata_df[(metadata_df['data_file_name'].str.contains('Hepatitis B'))].sort_values(by=['data_file_name','page_component'])

In [90]:
# file_subset_df.sort_values(by='data_file_name').reset_index(inplace=True)

In [99]:
# file_subset_df[file_subset_df['data_file_name'].str.contains("Hepatitis B")]['data_file_name'].unique()

In [8]:
i = 0

start_time = time.time()
new_time = time.time()

metadata_cols = ['start_year','end_year','last_updated','published_date','parquet_filename','data_file_url','doc_file_url','dataset']

for data_file_name, page_component in file_subset_df.sort_values(by='data_file_name').values:
    file_df = metadata_df[(metadata_df['data_file_name'] == data_file_name.strip()) &
                          (metadata_df['page_component'].str.contains(page_component.strip()))]
    if file_df['start_year'].max() > 2015:
        
        alias = generate_filename(data_file_name.lower() + " " + page_component.lower(),'')
        
        df = duckdb.read_parquet(f's3://{bucket_name}/all-continuous-nhanes/data/{alias}*.parquet',
                    union_by_name=True,
                    filename=True).to_df()
        
        df['survey'] = data_file_name
        df['survey_type'] = page_component
        if 'SEQN' in df.columns:
            df['SEQN'] = df['SEQN'].astype('Int64')  
        
        df['filename_only'] = df['filename'].apply(lambda x: x.split('/')[-1])
        
        df = df.merge(metadata_df[metadata_cols],
                 how='left',left_on='filename_only',right_on='parquet_filename')
        
        df['start_year'] = df['start_year'].astype('Int64')
        df['end_year'] = df['end_year'].astype('Int64')
        df['published_date'] = pd.to_datetime(df['published_date'],errors='ignore')
        
        df.drop(['filename_only'],axis=1, inplace=True)
        
        try:
            str_df = df.select_dtypes([object])
            str_df = str_df.stack().str.decode('utf-8').unstack()
        
            for col in str_df.columns:
                if col not in metadata_cols and col != 'filename':
                    df[col] = str_df[col]
        except Exception as ex:
            print(ex)
            print(f"Unable to convert data types from bytes to string for: {data_file_name} - {page_component}") 

        update_bq_table(
                        df,
                        alias,
                        dataset="nhanes",
                        bucket="nhanes_clean",
                        truncate=True,
                        max_error=0,
                        schema=None,
                    )
        i += 1
    
        if i % 10 == 0 and i > 0:
            print(f"Last 10 datasets took {time.time() - new_time} seconds")
            
print(f"Entire process took {time.time() - start_time} seconds")

acculturation_questionnaire_20231110_185042.csv uploaded to nhanes_clean / acculturation_questionnaire/
Starting job 09084aac-724b-4c28-a8ed-529c3769db19
Job finished.
Table Row Count 92158 rows.


/var/tmp/ipykernel_50151/2626580201.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['published_date'] = pd.to_datetime(df['published_date'],errors='ignore')


albumin_creatinine_urine_laboratory_20231110_185055.csv uploaded to nhanes_clean / albumin_creatinine_urine_laboratory/
Starting job 966ac994-c854-403d-9aea-9e2dd16b4824
Job finished.
Table Row Count 94940 rows.
alcohol_use_questionnaire_20231110_185109.csv uploaded to nhanes_clean / alcohol_use_questionnaire/
Starting job 2424b5d0-7daa-48ad-be84-e0c0fc86a364
Job finished.
Table Row Count 62524 rows.
alpha_1_acid_glycoprotein_serum_surplus_laboratory_20231110_185116.csv uploaded to nhanes_clean / alpha_1_acid_glycoprotein_serum_surplus_laboratory/
Starting job 030ee158-990f-4175-834f-ab20279d4cb6
Job finished.
Table Row Count 6002 rows.


/var/tmp/ipykernel_50151/2626580201.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['published_date'] = pd.to_datetime(df['published_date'],errors='ignore')


arsenic_total_urine_laboratory_20231110_185121.csv uploaded to nhanes_clean / arsenic_total_urine_laboratory/
Starting job 194cebe8-cfa8-43d7-b17e-4136f109e245
Job finished.
Table Row Count 13903 rows.


...

vitamin_c_laboratory_20231110_192059.csv uploaded to nhanes_clean / vitamin_c_laboratory/
Starting job 3b666405-87af-442d-9e2d-f473167155f6
Job finished.
Table Row Count 23503 rows.


/var/tmp/ipykernel_50151/2626580201.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['published_date'] = pd.to_datetime(df['published_date'],errors='ignore')


volatile_organic_compound_voc_metabolites_urine_laboratory_20231110_192105.csv uploaded to nhanes_clean / volatile_organic_compound_voc_metabolites_urine_laboratory/
Starting job bc94279b-bff9-42ed-869f-a51685eb3260
Job finished.
Table Row Count 16454 rows.
Last 10 datasets took 1835.585443496704 seconds
volatile_organic_compound_voc_metabolites_ii_urine_laboratory_20231110_192113.csv uploaded to nhanes_clean / volatile_organic_compound_voc_metabolites_ii_urine_laboratory/
Starting job 800726cb-bbac-4c4c-ac7a-cb449aa03eae
Job finished.
Table Row Count 4890 rows.
volatile_organic_compound_voc_metabolites_ii_urine_surplus_laboratory_20231110_192117.csv uploaded to nhanes_clean / volatile_organic_compound_voc_metabolites_ii_urine_surplus_laboratory/
Starting job ec8578e6-d481-4f5d-94d6-9b88b098ae77
Job finished.
Table Row Count 2979 rows.


/var/tmp/ipykernel_50151/2626580201.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['published_date'] = pd.to_datetime(df['published_date'],errors='ignore')


volatile_organic_compounds_and_trihalomethanes_mtbe_blood_laboratory_20231110_192122.csv uploaded to nhanes_clean / volatile_organic_compounds_and_trihalomethanes_mtbe_blood_laboratory/
Starting job 94ebc1ee-7c96-421c-b574-796730981bd7
Job finished.
Table Row Count 15251 rows.
volatile_toxicant_questionnaire_20231110_192134.csv uploaded to nhanes_clean / volatile_toxicant_questionnaire/
Starting job 85ae842b-0409-47db-a37d-50c2f3870da3
Job finished.
Table Row Count 11762 rows.
weight_history_questionnaire_20231110_192145.csv uploaded to nhanes_clean / weight_history_questionnaire/
Starting job bd3fcaec-c06e-42de-b21a-fbb8730029e5
Job finished.
Table Row Count 73787 rows.
weight_history_youth_questionnaire_20231110_192159.csv uploaded to nhanes_clean / weight_history_youth_questionnaire/
Starting job c020ea7e-0df2-4518-bb60-3196eff4edd8
Job finished.
Table Row Count 12777 rows.
Entire process took 1885.296294927597 seconds


In [ ]:
# str_df.values[2][1].decode('utf-8')

In [10]:
# df

In [24]:
col

'DRXFCSD'

In [15]:
# file_df['gcs_data_filename'].apply(lambda x: x.split('all_continuous')[0] + '*.parquet')

226     acculturation_questionnaire_*.parquet
449     acculturation_questionnaire_*.parquet
534     acculturation_questionnaire_*.parquet
1049    acculturation_questionnaire_*.parquet
1077    acculturation_questionnaire_*.parquet
1164    acculturation_questionnaire_*.parquet
1187    acculturation_questionnaire_*.parquet
1329    acculturation_questionnaire_*.parquet
1349    acculturation_questionnaire_*.parquet
1395    acculturation_questionnaire_*.parquet
1475    acculturation_questionnaire_*.parquet
Name: gcs_data_filename, dtype: object

In [57]:
df

,SEQN,ACD010A,ACD010B,ACD010C,ACQ020,ACQ030,ACD040,ACQ050,ACQ060,ACD070,ACD080,ACD011A,ACD011B,ACD011C,ACD110,filename
0,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
1,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
2,6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
3,7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
4,8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92153,124817,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
92154,124818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
92155,124820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...
92156,124821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,s3://nhanes_clean/all-continuous-nhanes/data/a...


In [59]:
# metadata_df['gcs_data_filename'].tolist()

In [56]:
# metadata_df[metadata_df['gcs_data_filename'].str.contains('audiometry_all_continuous_nhanes_1999_2000_data.parquet')]

In [149]:
file_df.sort_values(by=['start_year','end_year','data_file_name'], ascending=False).drop_duplicates(subset=['gcs_data_filename'])['gcs_data_filename'].tolist()



['physical_activity_monitor_raw_data_80hz_examination_all_continuous_nhanes_2013_2014_data.XPT',
 'physical_activity_monitor_raw_data_80hz_examination_all_continuous_nhanes_2011_2012_data.XPT']

,Success


In [ ]:
%%sql
output_df << SELECT * FROM read_parquet(f's3://nhanes/all-continuous-nhanes/data/audiometry_all*.parquet',
                                           union_by_name=True,
                                           filename=true);

In [44]:
# output_df['filename_only'] = output_df['filename'].apply(lambda x: x.split('/')[-1])

In [54]:
# output_df['filename_only'].tolist()

In [61]:
output_df.merge(metadata_df[['start_year','end_year','last_updated','published_date','parquet_filename']],how='left',left_on='filename_only',right_on='parquet_filename')




,SEQN,AUQ130,AUQ140,AUQ150,AUQ160,AUQ170,AUQ180,AUQ190,AUQ200,AUQ210,...,AUQ490,AUQ500,AUQ510,filename,filename_only,start_year,end_year,last_updated,published_date,parquet_filename
0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:40.087858+00:00,2005-02-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:42.493038+00:00,2002-06-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
2,2.0,2.0,4.0,2.0,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:40.087858+00:00,2005-02-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
3,2.0,2.0,4.0,2.0,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:42.493038+00:00,2002-06-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
4,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:40.087858+00:00,2005-02-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204049,124820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:42.493038+00:00,2021-09-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
204050,124821.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:40.087858+00:00,2022-03-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
204051,124821.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:42.493038+00:00,2021-09-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
204052,124822.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:40.087858+00:00,2022-03-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
